<a href="https://colab.research.google.com/github/elhamsh93/3D-vision/blob/main/PointCloudTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install path.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

import torchmetrics
from path import Path

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

--2023-05-18 08:48:01--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.74
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip [following]
--2023-05-18 08:48:01--  https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip.1’

ModelNet10.zip.1     27%[====>               ] 124.80M  12.5MB/s    eta 30s    ^C


In [ ]:
!unzip -q ModelNet10.zip

replace ModelNet10/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Load Dataset

In [ ]:
path = Path('ModelNet10')
path

Path('ModelNet10')

In [ ]:
path/'bed'

Path('ModelNet10/bed')

Read Off Files

In [ ]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [ ]:
with open(path/'chair/train/chair_0001.off', 'r') as f:
  verts, faces = read_off(f)

i, j, k = np.array(faces).T
x, y, z = np.array(verts).T
len(x)

2382

Visualize

In [ ]:
def rotateable_fig(data):
  
  x_eye = -1.25
  y_eye = 2
  z_eye = 0.5

  layout = go.Layout(
          title='Animation Test',
          width=600,
          height=600,
          scene=dict(camera=dict(eye=dict(x=x_eye, y=y_eye, z=z_eye))),
          updatemenus=[dict(type='buttons',
                    showactive=False,
                    y=1,
                    x=0.8,
                    xanchor='left',
                    yanchor='bottom',
                    pad=dict(t=45, r=10),
                    buttons=[dict(label='Play',
                                  method='animate',
                                  args=[None, dict(frame=dict(duration=2, redraw=False), 
                                                              transition=dict(duration=0),
                                                              fromcurrent=True,
                                                              mode='immediate'
                                                              )]
                                              )
                                        ]
                                )
                          ]
  )


  def rotate_z(x, y, z, theta):
      w = x+1j*y
      return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

  frames=[]
  for t in np.arange(0, 6.26, 0.1):
      xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
      frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))

  fig = go.Figure(data=data, layout=layout, frames=frames)

  return fig

In [ ]:
rotateable_fig([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i, j=j, k=k)]).show()

In [ ]:
def pcshow(xs, ys, zs):
  data=[go.Scatter3d(x=xs, y=ys, z=zs,
                     mode='markers')]
  fig = rotateable_fig(data)
  fig.update_traces(marker=dict(size=2,
                                line=dict(width=2, color='DarkSlateGray')), selector=dict(mode='markers'))
  fig.show()

In [ ]:
pcshow(x, y, z)

Sample Points

https://github.com/nikitakaraevv/pointnet/blob/master/source/utils.py

In [ ]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size

    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        

    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces,
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points

In [ ]:
pointcloud = PointSampler(2048)((verts, faces))

In [ ]:
pcshow(*pointcloud.T)

In [ ]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud


In [ ]:
norm_pointcloud = Normalize()(pointcloud)

In [ ]:
pcshow(*norm_pointcloud.T)

Augmentation

In [ ]:
class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

In [ ]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)

In [ ]:
pcshow(*noisy_rot_pointcloud.T)


> ToTensor


In [ ]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

Transform

In [ ]:
def default_transforms():
  return transforms.Compose([
        PointSampler(1024),
        Normalize(),
        ToTensor()
    ])
  

In [ ]:
train_transforms = transforms.Compose([
        PointSampler(1024),
        Normalize(),
        RandRotation_z(),
        RandomNoise(),
        ToTensor()
    ])

In [ ]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud,
                'category': self.classes[category]}

In [ ]:
train_ds = PointCloudData(path, transform=train_transforms)
test_ds = PointCloudData(path, valid=True, folder='test', transform=default_transforms())

In [ ]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()}
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

In [ ]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(test_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
Sample pointcloud shape:  torch.Size([1024, 3])


In [ ]:
data = train_ds[2000]
data

{'pointcloud': tensor([[ 0.4063,  0.1378,  0.3297],
         [ 0.7013,  0.2452,  0.4394],
         [ 0.4002,  0.1198, -0.3976],
         ...,
         [ 0.7564,  0.2714,  0.2634],
         [-0.8557, -0.2854, -0.2395],
         [ 0.6507,  0.2240, -0.3609]], dtype=torch.float64),
 'category': 5}

In [ ]:
pcshow(*data['pointcloud'].T)

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=64)

In [ ]:
batch = next(iter(train_loader))
batch['pointcloud'].shape

torch.Size([32, 1024, 3])

In [ ]:
class PCT(nn.Module):
  def __init__(self,):
    super(PCT, self).__init__()
    #Input Embedding
    self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm1d(64)
    self.conv2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm1d(64)

    # self.mini1 = MiniPOintNet()
    # self.mini2 = MiniPOintNet()

  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x))) # LBR1
    x = F.relu(self.bn2(self.conv2(x))) # LBR2



In [ ]:
model = PCT()

In [ ]:
model(torch.rand(32, 3, 1024))

Model

In [ ]:
# !pip install "git+git://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib"

In [ ]:
!git clone https://github.com/erikwijmans/Pointnet2_PyTorch.git

fatal: destination path 'Pointnet2_PyTorch' already exists and is not an empty directory.


In [ ]:
!pip install -r /content/Pointnet2_PyTorch/requirements.txt


ERROR: Invalid requirement: './pointnet2_ops_lib/.' (from line 9 of /content/Pointnet2_PyTorch/requirements.txt)
Hint: It looks like a path. File './pointnet2_ops_lib/.' does not exist.


In [ ]:
!pip install /content/Pointnet2_PyTorch/pointnet2_ops_lib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./Pointnet2_PyTorch/pointnet2_ops_lib
  Preparing metadata (setup.py) ... done
  Created wheel for pointnet2-ops: filename=pointnet2_ops-3.0.0-cp310-cp310-linux_x86_64.whl size=10710328 sha256=23ca71d7d26bcdc79f03351a1d5b0ae8173fd0a5bc452df0094a78425eebdefd
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkmq_esx/wheels/67/60/62/d9c88989cf12cdac35d245851846cfee8531e1e862bc0d7449
Successfully built pointnet2-ops
  Attempting uninstall: pointnet2-ops
    Found existing installation: pointnet2-ops 3.0.0
    Uninstalling pointnet2-ops-3.0.0:
      Successfully uninstalled pointnet2-ops-3.0.0


In [ ]:
import pointnet2_ops
from pointnet2_ops.pointnet2_utils import furthest_point_sample

# Sampling & Grouping

In [ ]:
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

In [ ]:
def knn_point(nsample, xyz, new_xyz):
    """
    Input:
        nsample: max sample number in local region
        xyz: all points, [B, N, C]
        new_xyz: query points, [B, S, C]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    sqrdists = square_distance(new_xyz, xyz)
    _, group_idx = torch.topk(sqrdists, nsample, dim=-1, largest=False, sorted=False)
    return group_idx

In [ ]:
def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points

In [ ]:
def sample_and_group(npoint, radius, nsample, xyz, points):
    """
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, npoint, nsample, 3]
        new_points: sampled points data, [B, npoint, nsample, 3+D]
    """
    B, N, C = xyz.shape
    S = npoint 
    xyz = xyz.contiguous()

    fps_idx = furthest_point_sample(xyz, npoint).long() # [B, npoint]
    new_xyz = index_points(xyz, fps_idx) 
    new_points = index_points(points, fps_idx)
    # new_xyz = xyz[:]
    # new_points = points[:]

    idx = knn_point(nsample, xyz, new_xyz)
    #idx = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = index_points(xyz, idx) # [B, npoint, nsample, C]
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, C)
    grouped_points = index_points(points, idx)
    grouped_points_norm = grouped_points - new_points.view(B, S, 1, -1)
    new_points = torch.cat([grouped_points_norm, new_points.view(B, S, 1, -1).repeat(1, 1, nsample, 1)], dim=-1)
    return new_xyz, new_points

# MiniPointNet

In [ ]:
class MiniPointNet(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(MiniPointNet, self).__init__()
    self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm1d(out_channels)
    self.conv2 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm1d(out_channels)
  
  def forward(self, x): # [[B, npoint, nsample, 3+D]] = torch.Size([32, 512, 32, 128])
    b, n, s, c = x.size()
    x = x.permute(0, 1, 3, 2) # [32, 512, 128, 32]
    x = x.reshape(-1, c, s) # [32*512, 128, 32]
    x = F.relu(self.bn1(self.conv1(x))) # LBR1
    x = F.relu(self.bn2(self.conv2(x))) # LBR2
    x = F.adaptive_max_pool1d(x, 1) # [32*512, 128]
    x = x.reshape(b, n, c) # [32, 512, 128]
    # x = x.permute(0, 2, 1)
    return x

# Self Attention

In [63]:
class Attention(nn.Module):
  def __init__(self, channels):
    super(Attention, self).__init__()
    self.q_conv = nn.Conv1d(channels, channels//4, 1, bias=False)
    self.k_conv = nn.Conv1d(channels, channels//4, 1, bias=False)
    self.v_conv = nn.Conv1d(channels, channels, 1, bias=False)
    self.softmax = nn.Softmax(dim=-1)
    self.conv = nn.Conv1d(channels, channels, 1)
    self.bn = nn.BatchNorm1d(channels)

  def forward(self, x):
    x_q = self.q_conv(x).permute(0, 2, 1)
    x_k = self.k_conv(x)
    x_v = self.v_conv(x)
    energy = torch.bmm(x_q, x_k)
    w = self.softmax(energy)
    w = w / (1e-6 + w.abs().sum(dim=1, keepdim=True))
    x_r = torch.bmm(x_v, w)
    x_r = F.relu(self.bn(self.conv(x - x_r)))
    x = x + x_r
    return x


In [64]:
class Transformer(nn.Module):
  def __init__(self, channels=256):
    super(Transformer, self).__init__()
    self.conv1 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)
    self.conv2 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)

    self.bn1= nn.BatchNorm1d(channels)
    self.bn2= nn.BatchNorm1d(channels)

    self.att1= Attention(channels)
    self.att2= Attention(channels)
    self.att3= Attention(channels)
    self.att4= Attention(channels)

  def forward(self, x):
    batch_size, _, N = x.size()
    x = F.relu(self.bn1(self.conv1(x))) 
    x = F.relu(self.bn2(self.conv2(x))) 
    x1 = self.att1(x)
    x2 = self.att2(x1)
    x3 = self.att3(x2)
    x4 = self.att4(x3)
    x = torch.cat((x1, x2, x3, x4), dim=1)
    return x


# PCT

In [65]:
class PCT(nn.Module):
  def __init__(self,):
    super(PCT, self).__init__()
    #Input Embedding
    self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm1d(64)
    self.conv2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm1d(64)

    self.mini1 = MiniPointNet(128, 128)
    self.mini2 = MiniPointNet(256, 256)

    # Transformer
    self.transformer = Transformer(256)
    # LBR
    self.conv3 = nn.Conv1d(in_channels=1024, out_channels=1024, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm1d(1024)
    # Classifier
    self.cls = nn.Sequential(nn.Linear(1024, 512),
                             nn.LayerNorm(512),
                             nn.LeakyReLU(negative_slope=0.2),
                             nn.Linear(512, 256),
                             nn.LayerNorm(256),
                             nn.LeakyReLU(negative_slope=0.2),
                             nn.Linear(256, 10))

  def forward(self, x):
    # x = torch.rand(32, 3, 1024)
    xyz = x.permute(0, 2, 1) #  [B, N, 3]
    x = F.relu(self.bn1(self.conv1(x))) # LBR1
    x = F.relu(self.bn2(self.conv2(x))) # LBR2
    x = x.permute(0, 2, 1) # [B, N, D]
    new_xyz, new_points = sample_and_group(npoint=512, radius=0.15, nsample=32, xyz=xyz, points=x)
    # new_points are 4d we should convert it to 3d
    new_points = self.mini1(new_points)
    new_xyz, new_points = sample_and_group(npoint=256, radius=0.2, nsample=32, xyz=new_xyz, points=new_points)
    new_points = self.mini2(new_points) # (b, n, c)
    feature = self.transformer(new_points)
    feature = F.relu(self.bn3(self.conv3(feature))) # LBR3
    feature = F.adaptive_max_pool1d(feature, 1).squeeze(-1)
    # Classifier
    feature = self.cls(feature)

    return feature



In [66]:
model = PCT().cuda()

In [67]:
feature = model(torch.rand(32, 3, 1024).cuda())

In [68]:
feature.shape

torch.Size([32, 10])

In [69]:
# new_points.shape #[32, 256, 256]

# Config

In [70]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [71]:
model.to(device)

PCT(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mini1): MiniPointNet(
    (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (mini2): MiniPointNet(
    (conv1): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affin

In [72]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [73]:
Loss = nn.CrossEntropyLoss()

In [74]:
class AverageMeter(object):
  def __init__(self):
    self.reset()
  
  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count +=n
    self.avg = self.sum / self.count

In [75]:
def train(model, train_loader, test_loader, epochs=15):
  for epoch in range(epochs):
    model.train()
    loss_total = AverageMeter()
    accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).cuda()

    for i, data in enumerate(train_loader, 0):
      inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
      optimizer.zero_grad()
      outputs = model(inputs.transpose(1, 2))
      loss = Loss(outputs, labels)
      loss.backward()
      optimizer.step()
      loss_total.update(loss)
      accuracy(outputs.softmax(dim=-1), labels)
      if i == 10:
        break
    acc = 100*accuracy.compute()

    model.eval()
    correct = total = 0
    with torch.no_grad():
      for i, data in enumerate(test_loader):
        inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
        outputs = model(inputs.transpose(1,2))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

In [76]:
torch.autograd.set_detect_anomaly(True) 

In [ ]:
train(model, train_loader, test_loader)